In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import time
import ast
import os
import json
import pandas as pd
import importlib
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from shutil import copyfile
from tqdm.notebook import tqdm
from PIL import Image
from string import Template
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

# Reproduce Results
SEED = 1992
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
tqdm.pandas() # For Progress Bar


# 📖 GAN Introduction


![gan](https://learn-neural-networks.com/wp-content/uploads/2020/04/gan.jpg)

The core idea of a GAN is based on the "indirect" training through the discriminator, another neural network that is able to tell how much an input is "realistic", which itself is also being updated dynamically This basically means that the generator is not trained to minimize the distance to a specific image, but rather to fool the discriminator. This enables the model to learn in an unsupervised manner.(Wiki)<br>

>I have also presented the Neural Style Transfer [Notebook Here](https://www.kaggle.com/marcinstasko/cots-neuralstyle-transfer-pytorch-augumentation)

Easy Explanation:
* ***Dimensional noise vector:***   is just the random noise (dough) where the final image will come from. It act like seed for the generator network
* ***Generator***  Network with create an image from the random noise
* ***Discriminator*** Network witch classify Real/Fake

Generator tries to cheat discriminator to generate images with will be real<br>
In the same time Discriminator learn the "cheat" techniques and it is better to distinguish between Real/Fake



## Please Upvote if you find this Helpful
> This notebook shows how to use generator to generate the new images and handle the results [Unlimited COTS Generator -Pytorch GAN in action](https://www.kaggle.com/marcinstasko/unlimited-cots-generator-pytorch-gan-in-action)<br>
> Another augumentation method Neural Style transfer [COTS - NeuralStyle Transfer Pytorch Augumentation](https://www.kaggle.com/marcinstasko/cots-neuralstyle-transfer-pytorch-augumentation)

# 🔨 ETL Pipeline
>We have to extract the bboxes for the training. For this purpose we will create the pipeline to generate snippets

## Enviroment Creation

In [ ]:
# Create the Folder Structure and I/O Locations

TRAIN_PATH = "../input/tensorflow-great-barrier-reef/train_images"
GAN_TRAINING_PATH = "./GAN"
GAN_MODEL_PATH = f"{GAN_TRAINING_PATH}/Models"
GAN_IMAGES_PATH = f"{GAN_TRAINING_PATH}/Images"
BBOX_PATH = "./BBOX"
RAW_BBOX_PATH = f"{BBOX_PATH}/raw"

!mkdir {GAN_TRAINING_PATH}
!mkdir {GAN_MODEL_PATH}
!mkdir {GAN_IMAGES_PATH}
!mkdir {BBOX_PATH}
!mkdir {RAW_BBOX_PATH}


## 🧹 Data Preprocessing

In [ ]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_path(row):
    row['image_path'] = f'{TRAIN_PATH}/video_{row.video_id}/{row.video_frame}.jpg'
    return row

def csv_preparation(df,min_annots=1):
    # Taken only annotated photos
    df["num_bbox"] = df['annotations'].apply(lambda x: str.count(x, 'x'))

    df_train = df[df["num_bbox"]>=min_annots]

    #Annotations 
    df_train['annotations'] = df_train['annotations'].progress_apply(lambda x: ast.literal_eval(x))
    df_train['bboxes'] = df_train.annotations.progress_apply(get_bbox)

    #Images resolution
    df_train["width"] = 1280
    df_train["height"] = 720

    #Path of images
    df_train = df_train.progress_apply(get_path, axis=1)
    return df_train


In [ ]:
# Data Preparation
train_raw = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")

df_train = csv_preparation(train_raw,min_annots=1)

df_train.head()

# 📁 Generate Data for GAN


In [ ]:
def save_bbox(df, bb_path):
    annotion_id = 0
    for ann_row in df.itertuples():
        img = cv2.imread(ann_row.image_path)
        bbox_list = ann_row.bboxes
        
        for bbox in bbox_list:
            b_width = bbox[2]
            b_height = bbox[3]

            # some boxes in COTS are outside the image height and width
            if (bbox[0] + bbox[2] > 1280):
                b_width = bbox[0] - 1280 
            if (bbox[1] + bbox[3] > 720):
                b_height = bbox[1] - 720 

            box = img[bbox[1]:bbox[1]+b_height, bbox[0]:bbox[0]+b_width]

            annotion_id += 1
            try:
                cv2.imwrite(f'{RAW_BBOX_PATH}/{annotion_id}.jpg', box)
            except:
                pass

In [ ]:
# To save the time we will query the data to get urls where bbox exist
save_bbox(df_train.loc[df_train.num_bbox>0], BBOX_PATH)

In [ ]:
# Count how many files in folder
!ls {RAW_BBOX_PATH} |wc -l

# 🚅 Training Procedure


## ⚙ Training Configuration

In [ ]:
# Parameters to define the model.
params = {
    "bsize" :   16,        # Batch size during training.
    'imsize' :  64,       # Spatial size of training images. All images will be resized to this size during preprocessing.
    'nc' :      3,        # Number of channles in the training images. For coloured images this is 3.
    'nz' :      100,      # Size of the Z latent vector (the input to the generator).
    'ngf' :     64,       # Size of feature maps in the generator. The depth will be multiples of this.
    'ndf' :     64,       # Size of features maps in the discriminator. The depth will be multiples of this.
    'nepochs' : 75,      # Number of training epochs.
    'lr' :      0.0001,   # Learning rate for optimizers
    'beta1' :   0.5,      # Beta1 hyperparam for Adam optimizer
    'save_epoch' : 5}     # Save step.

In [ ]:
device =torch.device('cuda')
random_transforms = [transforms.RandomRotation(degrees=5)]

transform = transforms.Compose([
                                transforms.Resize(params['imsize']),
                                transforms.CenterCrop(params['imsize']),
                                transforms.RandomApply(random_transforms, p=0.3),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
                                ])
train_data = torchvision.datasets.ImageFolder(BBOX_PATH, transform=transform)

dataloader = torch.utils.data.DataLoader(train_data,
                                          shuffle=True,
                                          batch_size=params['bsize']
                                         )

imgs, label = next(iter(dataloader))
imgs = imgs.numpy().transpose(0, 2, 3, 1)


In [ ]:
# Generate sample batch
sample_batch = next(iter(dataloader))

# Plot the training images.
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(
    sample_batch[0].to(device)[ : params['bsize']], padding=2, normalize=True).cpu(), (1, 2, 0)))

plt.show()

## 🔭 Create Generator and Discriminator instances

In [ ]:
def weights_init(w):
    """
    Initializes the weights of the layer, w.
    """
    classname = w.__class__.__name__
    if classname.find('conv') != -1:
        nn.init.normal_(w.weight.data, 0.0, 0.02)
    elif classname.find('bn') != -1:
        nn.init.normal_(w.weight.data, 1.0, 0.02)
        nn.init.constant_(w.bias.data, 0)

# Define the Generator Network
class Generator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Input is the latent vector Z.
        self.tconv1 = nn.ConvTranspose2d(params['nz'], params['ngf']*8,
            kernel_size=4, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(params['ngf']*8)

        # Input Dimension: (ngf*8) x 4 x 4
        self.tconv2 = nn.ConvTranspose2d(params['ngf']*8, params['ngf']*4,
            4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(params['ngf']*4)

        # Input Dimension: (ngf*4) x 8 x 8
        self.tconv3 = nn.ConvTranspose2d(params['ngf']*4, params['ngf']*2,
            4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(params['ngf']*2)

        # Input Dimension: (ngf*2) x 16 x 16
        self.tconv4 = nn.ConvTranspose2d(params['ngf']*2, params['ngf'],
            4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(params['ngf'])

        # Input Dimension: (ngf) * 32 * 32
        self.tconv5 = nn.ConvTranspose2d(params['ngf'], params['nc'],
            4, 2, 1, bias=False)
        #Output Dimension: (nc) x 64 x 64

    def forward(self, x):
        x = F.relu(self.bn1(self.tconv1(x)))
        x = F.relu(self.bn2(self.tconv2(x)))
        x = F.relu(self.bn3(self.tconv3(x)))
        x = F.relu(self.bn4(self.tconv4(x)))

        x = F.tanh(self.tconv5(x))

        return x

In [ ]:
# Define the Discriminator Network
class Discriminator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Input Dimension: (nc) x 64 x 64
        self.conv1 = nn.Conv2d(params['nc'], params['ndf'],
            4, 2, 1, bias=False)

        # Input Dimension: (ndf) x 32 x 32
        self.conv2 = nn.Conv2d(params['ndf'], params['ndf']*2,
            4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(params['ndf']*2)

        # Input Dimension: (ndf*2) x 16 x 16
        self.conv3 = nn.Conv2d(params['ndf']*2, params['ndf']*4,
            4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(params['ndf']*4)

        # Input Dimension: (ndf*4) x 8 x 8
        self.conv4 = nn.Conv2d(params['ndf']*4, params['ndf']*8,
            4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(params['ndf']*8)

        # Input Dimension: (ndf*8) x 4 x 4
        self.conv5 = nn.Conv2d(params['ndf']*8, 1, 4, 1, 0, bias=False)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2, True)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2, True)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2, True)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.2, True)

        x = torch.sigmoid(self.conv5(x))

        return x

In [ ]:
# Create the generator.
netG = Generator(params).to(device)

# Apply the weights_init() function to randomly initialize all
# weights to mean=0.0, stddev=0.2
netG.apply(weights_init)
# Print the model.
print(netG)

# Step 2. Create Discriminator
netD = Discriminator(params).to(device)
# Apply the weights_init() function to randomly initialize all

netD.apply(weights_init)
# Print the model.
print(netD)

# Binary Cross Entropy loss function.
criterion = nn.BCELoss()

#Create the fixed noise
fixed_noise = torch.randn(params['ngf'], params['nz'], 1, 1, device=device)

real_label = 1 # You can reduce it to 0.9 or less to more in depth control the training
fake_label = 0

# Optimizer for the discriminator.
optimizerD = optim.Adam(netD.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))
# Optimizer for the generator.
optimizerG = optim.Adam(netG.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))

## 📈 Training Loop

In [ ]:
# # Stores generated images as training progresses.
img_list = []
# # Stores generator losses during training.
G_losses = []
# # Stores discriminator losses during training.
D_losses = []
iters = 0
print("Gladiators let`s fight:")
print("#"*25)

for epoch in range(params['nepochs']):
    for i, data in enumerate(dataloader, 0):
        # Transfer data tensor to GPU/CPU (device)
        real_data = data[0].to(device)
        
        # Get batch size. Can be different from params['nbsize'] for last batch in epoch.
        b_size = real_data.size(0)
        
        # Make accumalated gradients of the discriminator zero.
        netD.zero_grad()
        
        # Create labels for the real data. (label=1)
        label = torch.full((b_size, ), real_label, device=device)
        output = netD(real_data).view(-1)
        output = output.to(torch.float32)
        label = label.to(torch.float32)
        errD_real = criterion(output, label)
        
        # Calculate gradients for backpropagation.
        errD_real.backward()
        D_x = output.mean().item()
        
        # Sample random data from a unit normal distribution.
        noise = torch.randn(b_size, params['nz'], 1, 1, device=device)
        # Generate fake data (images).
        fake_data = netG(noise)
        # Create labels for fake data. (label=0)
        label.fill_(fake_label  )

        output = netD(fake_data.detach()).view(-1)
        
        errD_fake = criterion(output, label)
        # Calculate gradients for backpropagation.
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        # Net discriminator loss.
        errD = errD_real + errD_fake
        # Update discriminator parameters.
        optimizerD.step()
        
        # Make accumalted gradients of the generator zero.
        netG.zero_grad()
        # We want the fake data to be classified as real. Hence
        # real_label are used. (label=1)
        label.fill_(real_label)
        # No detach() is used here as we want to calculate the gradients w.r.t.
        # the generator this time.
        output = netD(fake_data).view(-1)
        errG = criterion(output, label)
        # Gradients for backpropagation are calculated.
        # Gradients w.r.t. both the generator and the discriminator
        errG.backward()

        D_G_z2 = output.mean().item()
        # Update generator parameters.
        optimizerG.step()

        # Check progress of training.
        if i%250 == 0:
            print(torch.cuda.is_available())
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % ((epoch), params['nepochs'], i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save the losses for plotting.
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on a fixed noise.
        if (iters % 300 == 0) or ((epoch == params['nepochs']-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake_data = netG(fixed_noise).detach().cpu()
                vutils.save_image(real_data, f"{GAN_IMAGES_PATH}/real_samples.jpg", normalize=True)
                vutils.save_image(fake_data.data, f"{GAN_IMAGES_PATH}/fake_samples_epoch_{epoch}.jpg", normalize=True)
            img_list.append(vutils.make_grid(fake_data.data, padding=2, normalize=True))
        iters +=1
        

    # Save the model.
    if epoch % params['save_epoch'] == 0:
        torch.save({
            'generator' : netG.state_dict(),
            'discriminator' : netD.state_dict(),
            'optimizerG' : optimizerG.state_dict(),
            'optimizerD' : optimizerD.state_dict(),
            'params' : params
            },f"{GAN_MODEL_PATH}/model_epoch_{epoch}.pth")

# 🔍 Training History

In [ ]:
# Plot the training losses.
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Save the final trained model.
torch.save({
            'generator' : netG.state_dict(),
            'discriminator' : netD.state_dict(),
            'optimizerG' : optimizerG.state_dict(),
            'optimizerD' : optimizerD.state_dict(),
            'params' : params
            }, './model_final.pth')

# ✨ Animate training progress

In [ ]:
%matplotlib inline
import matplotlib.animation as animation

# Create the gif to show how training was performing :) 
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
anim = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=250, blit=True)
anim.save('gan.gif', dpi=35, writer='imagemagick')

In [ ]:
def show_gif(fname):
    import base64
    from IPython import display
    with open(fname, 'rb') as fd:
        b64 = base64.b64encode(fd.read()).decode('ascii')
    return display.HTML(f'<img src="data:image/gif;base64,{b64}" />')


show_gif("./gan.gif")

## Please Upvote if you find this Helpful
> This notebook shows how to use generator to generate the new images and handle the results [Unlimited COTS Generator -Pytorch GAN in action](https://www.kaggle.com/marcinstasko/unlimited-cots-generator-pytorch-gan-in-action)<br>
> Another augumentation method Neural Style transfer [COTS - NeuralStyle Transfer Pytorch Augumentation](https://www.kaggle.com/marcinstasko/cots-neuralstyle-transfer-pytorch-augumentation)

# ✂️ Remove Files

In [ ]:
!rm -r {GAN_TRAINING_PATH}
!rm -r {RAW_BBOX_PATH}